In [ ]:
!pip install g4f
!pip install openai
!pip install tqdm

In [ ]:
import g4f
import openai


OPENAI_TOKEN = '' # Токен с openai.com
SYSTEM_PROMPT = "Пожалуйста, проанализируйте исходный текст с целью выявления нецензурных, оскорбительных или недопустимых слов и выражений. Замените их на соответствующие, общепринятые и социально приемлемые синонимы или фразы, сохраняя при этом структурную и смысловую целостность текста. Не удаляйте или не сокращайте какие-либо элементы исходного текста; цель состоит в том, чтобы сделать его приемлемым для широкой аудитории. Например, слово 'неприличное выражение 1' может быть заменено на 'соответствующий синоним 1', слово 'неприличное выражение 2' на 'соответствующий синоним 2', и так далее."

# Метод добавления цензуры (если есть API_TOKEN)
def make_gpt_request(prompt, api_key = OPENAI_TOKEN) -> str:
    openai.api_key = api_key

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"{SYSTEM_PROMPT}"},
            {"role": "system", "content": f"{prompt}"},
        ]
    )

    generated_text = response["choices"][0]["message"]["content"]
    
    return generated_text

# Метод добавления цензуры (без токена)
def make_gpt_free_request(prompt: str) -> str:
    response = g4f.ChatCompletion.create(
        model=g4f.models.default,
        messages=[
            {"role": "system", "content": f"{SYSTEM_PROMPT}"},
            {"role": "user", "content": f"{prompt}"},
        ]
    )

    

    return response


make_censored_request = make_gpt_request if OPENAI_TOKEN else make_gpt_free_request # делаем bind метода, в зависимости от токена


In [ ]:
from tqdm import tqdm
import json

def get_file_as_dict(name: str) -> dict:
    with open(f'{name}', 'r') as f:
        return json.loads(f.read())

DATASET_PATH = './dataset.json'
# Теперь переменная 'data' содержит массив с данными формата [{title: название трека, text: текст песни} , ...]
data = get_file_as_dict(DATASET_PATH)

for idx, item in tqdm(enumerate(data)):
    item['censored'] = make_censored_request(item['text']) # делаем цензурирование с помощью ChatGPT
    output_file = f'./processed/{idx}.json' # привязываем название аутпут файла
    print(item['censored'])
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(item, f, ensure_ascii=False, indent=4)


In [1]:
## Посхалка ##
# Представьте, что вы Seinor ChatGPT - Prompt Engineer, ваша задача как реального интеллекта, 
# решить все этапы препроцессинга текста с использованием
# ChatGPT API -> что нужно?
## 0.0 Написать классный промпт, который делает:
## 1) lowercase
## 2) отбросить стоп слова + не текстовые символы 
## 3) перевод в нормальную форму 
## 4) когда текст иностранный -> приведение к единому языку (например русский)

# Правила игры:
# Первый кто сделает задачу - получает 5 доп. баллов
# Второй - 4 балла
# Третий - 3 балла
# и так далее

# Что можно сделать с ними? 
# 1) оставить себе 💅
# 2) отдать другу 👩‍❤‍👨
# 3) продать на черном рынке 😈